In [1]:
from email import message
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
import csv 
import pandas as pd
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

if os.path.exists('token.pickle'):
  
        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

service = build('gmail', 'v1', credentials=creds)
query ='from:no-reply@thecitybank.com "transaction alert"'
result = service.users().messages().list(userId='me',q=query).execute()
messages = []
if 'messages' in result:
        messages.extend(result['messages'])
while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])

In [8]:
messages_df = pd.DataFrame(messages)

In [13]:
messages_df[messages_df['threadId']=='181bf769af9eeb75']

,id,threadId


In [ ]:
# ex_thread:
# FMfcgzGpGnGwnQKrkrwvbdtMtRqbwsWP

# "id": "181bf92ecf39008d",
#       "threadId": "181bf769af9eeb75"

In [ ]:
msg_rn = service.users().messages().get(userId='me', id='1827422df69315a4').execute()

In [ ]:
# this contains the message id. search using>: rfc822msgid:
msg_rn['payload']['headers'][17]['value']

In [ ]:
# this contains the date of delivery
msg_rn['payload']['headers'][12]

In [2]:
dfs = []
for msg in messages: 
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    payload = txt['payload']
    msg_id = payload['headers'][17]['value']
    msg_time = payload['headers'][12]['value']

    if 'parts' in payload.keys():
                parts = payload.get('parts')[1]
                data = parts['body']['data']
                data = data.replace("-","+").replace("_","/")
                decoded_data = base64.b64decode(data)
                soup = BeautifulSoup(decoded_data , "html.parser")
                # body = soup.body()
                text = []
                for tr in soup.find_all('table')[1].find_all('table')[3].find_all('tr'):
                    for td in tr.find_all('td'):
                        text.append(str(td.string))
                keys_ = text[::2]
                values_ = text[1::2]
                dict_ = dict(zip(keys_,values_))
                temp_df = pd.DataFrame([dict_])
                temp_df['msg_id']= msg_id
                temp_df['msg_time'] = msg_time
                temp_df['id']=msg['id']
                dfs.append(temp_df)


                
# keys_ = text[::2]
# values_ = text[1::2]
# dict_ = dict(zip(keys_,values_))                

In [3]:
df = pd.concat(dfs).reset_index()

In [ ]:
df[['Transaction Amount','Available Balance']].apply(lambda x:x.str.replace('BDT',''))

In [ ]:
df.id

In [7]:
pd.DataFrame(messages).to_clipboard()

In [4]:
df.to_csv('output_1.csv',index=False)

In [ ]:
import pandas as pd
dict_ = dict(zip(list(enumerate(keys_)),list(enumerate(values_))))

In [ ]:
pd.DataFrame.from_dict(dict_,orient='index')